In [1]:
import pandas as pd
import requests
from azure.storage.blob import BlobServiceClient
import io
import pyarrow.parquet as pq
import pyarrow as pa
import json
import os

# Credenciais
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name_send = 'gold'

def extrair_dados_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['dados']
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None

def transformar_dados_para_dataframe(dados):
    df = pd.json_normalize(dados)
    return df

def enviar_parquet_azure(account_name, account_key, container_name_send, file_name, dataframe):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_send)

    # Converta DataFrame para tabela Parquet
    table = pa.Table.from_pandas(dataframe)

    # Crie um buffer de memória para escrever os dados Parquet
    parquet_buffer = io.BytesIO()
    pq.write_table(table, parquet_buffer)

    # Envie os dados para o Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)

    print(f"Arquivo Parquet '{file_name}' enviado com sucesso para o Azure Blob Storage.")

def processar_dados_deputados(account_name, account_key, container_name_send):
    url = "https://dadosabertos.camara.leg.br/api/v2/deputados"
    dados = extrair_dados_api(url)
    if dados:
        df = transformar_dados_para_dataframe(dados)
        enviar_parquet_azure(account_name, account_key, container_name_send, 'deputadosPartidos.parquet', df)
        print("Processamento dos dados de deputados concluído.")

# Processar dados dos deputados
processar_dados_deputados(account_name, account_key, container_name_send)


Arquivo Parquet 'deputadosPartidos.parquet' enviado com sucesso para o Azure Blob Storage.
Processamento dos dados de deputados concluído.
